In [ ]:
!pip3 install transformers

In [2]:
import numpy as np
import pandas as pd

import torch
from torch.utils.data import Dataset, DataLoader

from tqdm import tqdm

from transformers import AutoModelWithLMHead, AutoTokenizer, pipeline

In [3]:
assert torch.cuda.is_available(), 'у вас не находится гпу'

device = torch.device('cuda:0')

In [ ]:
!unzip "aneqs.zip" # здесь нужно указать путь к архиву с csv-файлом

In [6]:
aneqs = pd.read_csv('aneqs.csv', encoding='cp1251')
aneqs.text = aneqs.text.map(lambda x: x.lower())

In [7]:
data_valid = ''
data_train = ''
data_len = len(aneqs)
for text in aneqs.text:
  text = text + '\n'
  if len(data_train) <= 600000:
    data_train+= text
  elif len(data_valid) <= 20000:
    data_valid +=text
  else:
    break


with open('test_file.txt', 'w', encoding='utf8') as f2:
  f2.write(data_valid)
  

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("sberbank-ai/rugpt3small_based_on_gpt2")

In [10]:
tokenizer.pad_token = 0
tokenizer.bos_token = 2
tokenizer.eos_token = 3

In [ ]:
!unzip 'AneqGPT_model.zip' #путь к архиву с моделью

In [ ]:
aneq_model=AutoModelWithLMHead.from_pretrained('./gpt3-aneqs').to(device)

In [13]:
with open('test_file.txt', encoding='utf-8') as f:
  content = f.readlines()

In [14]:
encodings = tokenizer(content, return_tensors='pt', padding=True)

In [15]:
max_length = aneq_model.config.n_positions
stride = 512

lls = []
for i in tqdm(range(0, encodings.input_ids.size(1), stride)):
    begin_loc = max(i + stride - max_length, 0)
    end_loc = min(i + stride, encodings.input_ids.size(1))
    trg_len = end_loc - i   
    input_ids = encodings.input_ids[:,begin_loc:end_loc].to(device)
    target_ids = input_ids.clone()
    target_ids[:,:-trg_len] = -100

    with torch.no_grad():
        outputs = aneq_model(input_ids, labels=target_ids)
        log_likelihood = outputs[0] * trg_len

    lls.append(log_likelihood)

ppl = torch.exp(torch.stack(lls).sum() / end_loc)

100%|██████████| 1/1 [00:01<00:00,  1.78s/it]


In [16]:
print(ppl)

tensor(32.0701, device='cuda:0')
